In [ ]:
import pyvisa

rm = pyvisa.ResourceManager()
print (rm.list_resources())

TC290 = rm.open_resource("ASRL3::INSTR")
TC290.baud_rate = 115200
print (TC290.query('*IDN?'))

TC290.write("*rst; status:preset; *cls")

In [ ]:
def get_temp():
    while(1):
        temp = TC290.query("KRDG? A")
        print (temp)

def set_temp(temp):
    TC290.write("SETP 1, %.4f" % temp)